# <span style="color:#0b486b">  Student Information</span>
***
Surname: **Zhang**  <br/>
Firstname: **Yiming**    <br/>
Student ID: **35224436**    <br/>
Email: **yzha1213@student.monash.edu**    <br/>
Your tutorial time: **Wed 12pm**    <br/>
***

## <span style="color:#0b486b">Part 2: Deep Neural Networks (DNN) </span>
<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 25 points]<span></div>

The second part of this assignment is to demonstrate your basis knowledge in deep learning that you have acquired from the lectures and tutorials materials. Most of the contents in this assignment are drawn from **the tutorials covered from weeks 1 to 2**. Going through these materials before attempting this assignment is highly recommended.

In the second part of this assignment, you are going to work with the FashionMNIST dataset for the image recognition task. It has the exact same format as MNIST (70,000 grayscale images of 28 × 28 pixels each with 10 classes), but the images represent fashion items rather than handwritten digits, so each class is more diverse, and the problem is significantly more challenging than MNIST.

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

torch.manual_seed(1234)

**Load the Fashion MNIST using `torchvision`**

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset_orgin = datasets.FashionMNIST(root='./data', train=True, download=False, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=False, transform=transform)

print(train_dataset_orgin.data.shape, train_dataset_orgin.targets.shape)
print(test_dataset.data.shape, test_dataset.targets.shape)

# Flatten the data
train_dataset_orgin.data = train_dataset_orgin.data.reshape(-1, 28*28)
test_dataset.data = test_dataset.data.reshape(-1, 28*28)

print(train_dataset_orgin.data.shape, train_dataset_orgin.targets.shape)
print(test_dataset.data.shape, test_dataset.targets.shape)

N = len(train_dataset_orgin)
print(f"Number of training samples: {N}")
N_train = int(0.9*N)
N_val = N - N_train
print(f"Number of training samples: {N_train}")
print(f"Number of validation samples: {N_val}")

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_orgin, [N_train, N_val])


print(len(train_dataset))
print(len(val_dataset))


train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


####  <span style="color:red">**Question 2.1:**</span> Write the code to visualize a mini-batch in `train_loader` including its images and labels.

<div style="text-align: right"> <span style="color:red">[5 points]</span> </div>

In [ ]:
# Your code here
import matplotlib.pyplot as plt


def visualize_images(loader, batch_size=64):
    """
    Visualize the images in the loader
    batch_size: the number of images to show
    """
    images, labels = next(iter(loader))

    cols = int(batch_size**0.5)
    rows = (batch_size + cols - 1) // cols

    plt.figure(figsize=(12, 8))
    for idx in range(min(batch_size, len(images))):
        plt.subplot(rows, cols, idx + 1)
        # reshape the image to 28x28
        image = images[idx].detach().numpy().reshape(28, 28)
        plt.imshow(image, cmap="gray", interpolation="nearest")
        plt.title(f"Class: {labels[idx].item()}", fontsize=10)
        plt.axis("off")

    plt.suptitle("Visualization ofFashion-MNIST", fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()


visualize_images(train_loader)

#### <span style="color:red">**Question 2.2:**</span> Write the code for the feed-forward neural net using PyTorch

<div style="text-align: right"> <span style="color:red">[5 points]</span> </div>

We now develop a feed-forward neural network with the architecture $784 \rightarrow 40(ReLU) \rightarrow 30(ReLU) \rightarrow 10(softmax)$. You can choose your own way to implement your network and an optimizer of interest. You should train model in $50$ epochs and evaluate the trained model on the test set.

##### FFN Class

In [ ]:
# Your code here
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam


# ===== Model =====
class FFN(nn.Module):
    def __init__(self):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(784, 40)
        self.fc2 = nn.Linear(40, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        # flatten the image to 28*28
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = F.softmax(self.fc3(x), dim=1) # (B, 10)
        return logits



##### Initialize the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FFN().to(device)
model

##### Define loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss() # cross entropy loss
optimizer = Adam(model.parameters(), lr=1e-3) # Adam optimizer

##### Training function

In [ ]:
def run_epoch(model, loader, train=True, criterion=None, optimizer=None):
    model.train(mode=train)
    total_loss, total_correct, total_samp = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        if train:  # only update the model parameters when training
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samp += images.size(0)

    avg_loss = total_loss / total_samp
    acc = total_correct / total_samp
    return avg_loss, acc

##### Training process

In [ ]:
# ===== Training loop (50 epochs) =====
print("Training...\n")
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = run_epoch(
        model, train_loader, train=True, criterion=criterion, optimizer=optimizer
    )
    valid_loss, valid_acc = run_epoch(
        model, val_loader, train=False, criterion=criterion, optimizer=optimizer
    )

    print(
        f"Epoch {epoch}/{EPOCHS}, "
        f"Train loss {train_loss:.4f} acc {(train_acc*100):.2f}%, "
        f"Valid loss {valid_loss:.4f} acc {(valid_acc*100):.2f}%"
    )
print("Training done!")
# Test the model with test set
print(f"\nTesting Result:")
test_loss, test_acc = run_epoch(
    model,
    test_loader,
    train=False,
    criterion=criterion,
    optimizer=optimizer,
)
print(f"Test loss: {test_loss:.4f}, Valid acc: {(test_acc*100):.2f}")

####  <span style="color:red">**Question 2.3:**</span> Tuning hyper-parameters with grid search
<div style="text-align: right"> <span style="color:red">[5 points]</span> </div>


Assume that you need to tune the number of neurons on the first and second hidden layers $n_1 \in \{20, 40\}$, $n_2 \in \{20, 40\}$  and the used activation function  $act \in \{sigmoid, tanh, relu\}$. The network has the architecture pattern $784 \rightarrow n_1 (act) \rightarrow n_2(act) \rightarrow 10(softmax)$ where $n_1, n_2$, and $act$ are in their grides. Write the code to tune the hyper-parameters $n_1, n_2$, and $act$. Note that you can freely choose the optimizer and learning rate of interest for this task.

In [ ]:
# Your code here
class tunable_FFN(nn.Module):
    """FNN can be tuned by changing the number of neurons and the activation function"""

    def __init__(self, n1, n2, act):
        super(tunable_FFN, self).__init__()
        self.fc1 = nn.Linear(784, n1)
        self.fc2 = nn.Linear(n1, n2)
        self.fc3 = nn.Linear(n2, 10)
        self.act = act

    def forward(self, x):
        # flatten the image to 28*28
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = F.softmax(self.fc3(x), dim=1)  # (B, 10)
        return logits


EPOCHS_TUNE = 50
# store the results of each hyper-parameter combination (n1, n2, act, valid_loss, valid_acc, test_loss, test_acc)
results = []
best_config = None  # store the best config for question 2.5


def train_on_epoch(model, loader, criterion, optimizer, train=True):
    """Train the model for each epoch"""
    model.train(mode=train)
    loss_epoch, acc_epoch, samp_epoch = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_epoch += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        acc_epoch += (preds == labels).sum().item()
        samp_epoch += images.size(0)

    train_loss_epoch = loss_epoch / samp_epoch  # loss of the epoch
    train_acc_epoch = acc_epoch / samp_epoch  # accuracy of the epoch
    return train_loss_epoch, train_acc_epoch


def train_model(model, criterion, optimizer):
    """Train the model with different hyper-parameters"""
    for epoch in range(1, EPOCHS_TUNE + 1):
        train_loss, train_acc = train_on_epoch(
            model, train_loader, criterion, optimizer, train=True
        )
        valid_loss, valid_acc = train_on_epoch(
            model, test_loader, criterion, optimizer, train=False
        )
        print(
            f"Epoch {epoch}/{EPOCHS_TUNE}, "
            f"Train loss {train_loss:.4f} acc {(train_acc*100):.2f}%, "
            f"Valid loss {valid_loss:.4f} acc {(valid_acc*100):.2f}%"
        )

    return valid_loss, valid_acc


def evaluate(model=None):
    """test the model on the test set"""
    optimizer = Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    test_loss, test_acc = train_on_epoch(
        model, test_loader, criterion, optimizer, train=False
    )
    print(f"test_loss={test_loss:.4f}, test_acc={(test_acc*100):.2f}%")
    return test_loss, test_acc


def fine_tune():
    global best_config, results
    """Fine-tune the model with different hyper-parameters"""
    n1_grid = [20, 40]
    n2_grid = [20, 40]
    act_grid = [F.sigmoid, F.tanh, F.relu]
    print("Start tuning...")
    for n1 in n1_grid:
        for n2 in n2_grid:
            for act in act_grid:
                print(f"\n[Config] n1 = {n1}, n2 = {n2}, act = {act.__name__}")
                model = tunable_FFN(n1, n2, act).to(device)
                optimizer = Adam(model.parameters(), lr=1e-3)
                criterion = nn.CrossEntropyLoss()
                valid_loss, valid_acc = train_model(model, criterion, optimizer)
                test_loss, test_acc = evaluate(model)
                results.append(
                    (n1, n2, act, valid_loss, valid_acc, test_loss, test_acc)
                )
    print("Tuning done!")

    # find the best config and model by max test_acc
    best_config = max(results, key=lambda x: x[6])
    n1, n2, act, _, _, _, best_acc = best_config
    print(
        f"Best config:",
        f"\nNetwork architecture: 784 -> {n1} -> {n2} -> 10",
        f"\nActivation function: {act.__name__}",
        f"\nBest accuracy: {best_acc*100:.2f}%",
    )


fine_tune()

####  <span style="color:red">**Question 2.4:**</span> Implement the loss with the form: $loss(p,y)=CE(1_{y},p)+\lambda H(p)$ where $H(p)=-\sum_{i=1}^{M}p_{i}\log p_{i}$ is the entropy of $p$, $p$ is the prediction probabilities of a data point $x$ with the ground-truth label $y$, $1_y$ is an one-hot label, and $\lambda >0$ is a trade-off parameter. Set $\lambda = 0.1$ to train a model.

<div style="text-align: right"> <span style="color:red">[5 points]</span> </div>


In [ ]:
# Your code here
import torch.nn as nn
import torch.nn.functional as F


class CEWithRegularization(nn.Module):
    """CE with regularization item."""

    def __init__(self, lam=0.1):
        super(CEWithRegularization, self).__init__()
        self.lam = lam
        self.ce = nn.CrossEntropyLoss()

    def forward(self, logits, targets):
        ce = self.ce(logits, targets)
        p = F.softmax(logits, dim=1)  # convert logits to probability
        epsilon = 1e-12  # avoid log(0)
        entropy = -(p * (p + epsilon).log()).sum(dim=1).mean()  # average over batch

        return ce + self.lam * entropy  # total loss


class FFN_CER(nn.Module):
    """FFN with CE with regularization."""

    def __init__(self):
        super(FFN_CER, self).__init__()
        self.fc1 = nn.Linear(784, 40)
        self.fc2 = nn.Linear(40, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

##### Test

In [ ]:
model = FFN_CER()
criterion = CEWithRegularization()
optimizer = Adam(model.parameters(), lr=1e-3)
print("Start training with CE with regularization..\n")

# training model
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)

    mean_loss = total_loss / len(train_loader)

    # validation
    model.eval() # Set model to eval mode for inference
    total_correct = 0
    total_samp = 0
    with torch.no_grad(): # No need to track gradients for validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            preds = logits.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            total_samp += labels.size(0)
    val_acc = total_correct / total_samp
    print(
        f"Epoch {epoch}/{EPOCHS}, Train loss {mean_loss:.4f}, Valid acc {(val_acc*100):.2f}%"
    )


# test model
model.eval()
total_correct = 0
total_samp = 0

with torch.no_grad():
    for images, labels in test_loader:
        logits = model(images)
        preds = logits.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samp += labels.size(0)
test_acc = total_correct / total_samp
print(f"Test acc: {(test_acc*100):.2f}%")

####  <span style="color:red">**Question 2.5:**</span> Experimenting with **sharpness-aware minimization** technique
<div style="text-align: right"> <span style="color:red">[5 points]</span> </div>

Sharpness-aware minimization (SAM) (i.e., [link for main paper](https://openreview.net/pdf?id=6Tm1mposlrM) from Google Deepmind) is a simple yet but efficient technique to improve the generalization ability of deep learning models on unseen data examples. In your research or your work, you might potentially use this idea. Your task is to read the paper and implement *Sharpness-aware minimization (SAM)*. Finally, you need to apply SAM to the best architecture found in **Question 2.3**.   

In [ ]:
# Your code here
class SAM_optimizer(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        # kwargs 里放 base optimizer 的超参，如 lr/momentum/weight_decay
        defaults = dict(rho=rho, **kwargs)
        super().__init__(params, defaults)
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.rho = rho

    def grad_norm(self):
        device = self.param_groups[0]["params"][0].device
        norms = []
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad
                norms.append(grad.norm(p=2))
        return torch.norm(torch.stack(norms), p=2).to(device)

    @torch.no_grad()
    def first_step(self, zero_grad=True):
        scale = self.rho / (self.grad_norm() + 1e-12)
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                e = p.grad
                e = e * scale
                p.add_(e)  # w <- w + e
                self.state[p]["e_w"] = e  # store the perturbation
        if zero_grad:
            self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=True):
        """recover the parameters"""
        for group in self.param_groups:
            for p in group["params"]:
                if "e_w" in self.state[p]:
                    p.sub_(self.state[p]["e_w"])  # w <- w - e
        self.base_optimizer.step()  # update the model
        if zero_grad:
            self.zero_grad()

    def zero_grad(self):
        self.base_optimizer.zero_grad()

##### Test with best config

In [ ]:
class FFN_best(nn.Module):
    def __init__(self, n1, n2, act):
        super().__init__()
        self.fc1 = nn.Linear(784, n1)
        self.fc2 = nn.Linear(n1, n2)
        self.fc3 = nn.Linear(n2, 10)
        self.act = act

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        return self.fc3(x)  # logits


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the parameters from the best config
print("Best config:", best_config)
n1, n2, act, _, _, _, _ = best_config
model = FFN_best(n1, n2, act).to(device)
print("Best model:", model)
criterion = nn.CrossEntropyLoss()

sam = SAM_optimizer(
    model.parameters(),
    base_optimizer=torch.optim.SGD,
    rho=0.05,
    lr=0.01,
)


def train_one_epoch(loader):
    model.train()
    total, correct, total_loss = 0, 0, 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        # step 1
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        sam.first_step(zero_grad=True)

        # step 2
        logits_perturbed = model(images)
        loss_perturbed = criterion(logits_perturbed, labels)
        loss_perturbed.backward()
        sam.second_step(zero_grad=True)

        # statistics
        total += images.size(0)
        correct += logits.argmax(1).eq(labels).sum().item()
        total_loss += loss.item() * images.size(0)

    return total_loss / total, correct / total


@torch.no_grad()
def evaluate(loader):
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)
        total += images.size(0)
        correct += logits.argmax(1).eq(labels).sum().item()
        total_loss += loss.item() * images.size(0)
    return total_loss / total, correct / total


# training loop
print("Training with SAM...\n")
EPOCHS_SAM = 20
for ep in range(1, EPOCHS_SAM + 1):
    tr_loss, tr_acc = train_one_epoch(train_loader)
    va_loss, va_acc = evaluate(val_loader)
    print(
        f"Epoch {ep:02d}/{EPOCHS_SAM}, Train loss {tr_loss:.4f} acc {(tr_acc*100):.2f}%, Val loss {va_loss:.4f} acc {(va_acc*100):.2f}%"
    )
print("\nTraining with SAM done!")

# test model
test_loss, test_acc = evaluate(test_loader)
print(
    f"\nTest results with SAM -> Test loss: {test_loss:.4f}, Test acc: {(test_acc*100):.2f}%"
)